YT Vid: https://www.youtube.com/watch?v=cO5g5qLrLSo

# Import Libs

In [13]:
import random
import gym

In [14]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0]
actions = env.action_space.n

In [15]:
states

4

In [16]:
actions

2

### Random actions

In [17]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode {episode} Score: {score}")

Exception ignored in: <function Viewer.__del__ at 0x00000219E0009CF0>
Traceback (most recent call last):
  File "C:\Users\robin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\envs\classic_control\rendering.py", line 165, in __del__
    self.close()
  File "C:\Users\robin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\envs\classic_control\rendering.py", line 83, in close
    self.window.close()
  File "C:\Users\robin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyglet\window\win32\__init__.py", line 299, in close
    super(Win32Window, self).close()
  File "C:\Users\robin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyglet\window\__init__.py", line 823, in close
    app.wi

Episode 1 Score: 12.0
Episode 2 Score: 48.0
Episode 3 Score: 17.0
Episode 4 Score: 15.0
Episode 5 Score: 37.0
Episode 6 Score: 14.0
Episode 7 Score: 15.0
Episode 8 Score: 51.0
Episode 9 Score: 40.0
Episode 10 Score: 13.0


## Deep Learning Model

In [18]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [19]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(actions, activation="linear"))
    return model

In [20]:
model = build_model(states, actions)

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


## Reinforcementlearning agent

In [22]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [23]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=10000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    
    return dqn

In [24]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=["mae"])
dqn.fit(env, nb_steps=10000, visualize=False, verbose = 1)

Training for 1 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 4:30 - reward: 1.0000done, took 0.028 seconds


C:\Users\robin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [25]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history["episode_reward"]))

Testing for 100 episodes ...
Episode 1: reward: 86.000, steps: 86
Episode 2: reward: 37.000, steps: 37
Episode 3: reward: 115.000, steps: 115
Episode 4: reward: 95.000, steps: 95
Episode 5: reward: 45.000, steps: 45
Episode 6: reward: 101.000, steps: 101
Episode 7: reward: 54.000, steps: 54
Episode 8: reward: 34.000, steps: 34
Episode 9: reward: 106.000, steps: 106
Episode 10: reward: 69.000, steps: 69
Episode 11: reward: 49.000, steps: 49
Episode 12: reward: 58.000, steps: 58
Episode 13: reward: 59.000, steps: 59
Episode 14: reward: 39.000, steps: 39
Episode 15: reward: 63.000, steps: 63
Episode 16: reward: 57.000, steps: 57
Episode 17: reward: 143.000, steps: 143
Episode 18: reward: 94.000, steps: 94
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 130.000, steps: 130
Episode 21: reward: 42.000, steps: 42
Episode 22: reward: 56.000, steps: 56
Episode 23: reward: 61.000, steps: 61
Episode 24: reward: 68.000, steps: 68
Episode 25: reward: 77.000, steps: 77
Episode 26: reward: 

In [26]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 200 episodes ...
Episode 1: reward: 47.000, steps: 47
Episode 2: reward: 84.000, steps: 84
Episode 3: reward: 84.000, steps: 84
Episode 4: reward: 64.000, steps: 64
Episode 5: reward: 116.000, steps: 116
Episode 6: reward: 36.000, steps: 36
Episode 7: reward: 46.000, steps: 46
Episode 8: reward: 92.000, steps: 92
Episode 9: reward: 41.000, steps: 41
Episode 10: reward: 46.000, steps: 46
Episode 11: reward: 112.000, steps: 112
Episode 12: reward: 74.000, steps: 74
Episode 13: reward: 151.000, steps: 151
Episode 14: reward: 134.000, steps: 134
Episode 15: reward: 51.000, steps: 51
Episode 16: reward: 78.000, steps: 78
Episode 17: reward: 66.000, steps: 66
Episode 18: reward: 42.000, steps: 42
Episode 19: reward: 70.000, steps: 70
Episode 20: reward: 100.000, steps: 100
Episode 21: reward: 56.000, steps: 56
Episode 22: reward: 97.000, steps: 97
Episode 23: reward: 130.000, steps: 130
Episode 24: reward: 84.000, steps: 84
Episode 25: reward: 40.000, steps: 40
Episode 26: reward